In [5]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
from pymatreader import read_mat
from scipy import sparse
import numpy as np
import os
from datetime import datetime, timedelta
import pandas
import cv2
import json
import pandas as pd
from amftrack.pipeline.paths.directory import *
from amftrack.util import *
import pickle
from joblib import Parallel, delayed
from datetime import datetime
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.post_processing.global_plate import *
from amftrack.pipeline.functions.post_processing.global_hypha import *
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.post_processing.time_plate import *
from amftrack.pipeline.functions.post_processing.area_hulls import *
from amftrack.pipeline.functions.post_processing.exp_plot import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
API = str(np.load(os.getenv('HOME')+'/pycode/API_drop.npy'))
dir_drop = 'data_tables'
root = os.getenv('TEMP')
path_save = f'{root}time_plate_infos.pick'
download(API,f'/{dir_drop}/time_plate_infos.pick',path_save)
time_plate_info = pd.read_pickle(path_save)
path_save = f'{root}global_hypha_info.pick'
download(API,f'/{dir_drop}/global_hypha_info.pick',path_save)
global_hypha_info = pd.read_pickle(path_save)
path_save = f'{root}time_hypha_info.pick'
download(API,f'/{dir_drop}/time_hypha_info.pick',path_save)
time_hypha_info = pd.read_pickle(path_save)

ApiError: ApiError('329bde8a058941498c2da8c0e5c14df0', DownloadError('path', LookupError('not_found', None)))

In [2]:
%load_ext autoreload
%autoreload 2
directory = directory_project
# directory = '/projects/0/einf914/agg/'
# update_analysis_info(directory)
analysis_info = get_analysis_info(directory)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
analysis_info

,folder_analysis,version,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,...,Temp,date_begin,date_end,number_timepoints,path_exp,path_global_hypha_info,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis
0,Analysis_1636046826216022929_0_100_Version9,Version9,/projects/0/einf914/data/20211005_0148_Plate39,799,39,Carrot,A5,001P100N,Y,-,...,None,20211001_1620_Plate39,20211011_1227_Plate39,95,Analysis_1636046826216022929_0_100_Version9/ex...,Analysis_1636046826216022929_0_100_Version9/gl...,Analysis_1636046826216022929_0_100_Version9/ti...,Analysis_1636046826216022929_0_100_Version9/ti...,Analysis_1636046826216022929_0_100_Version9/gl...,"01.01.1970, 01:00:"
1,Analysis_1636107306199118080_0_100_Version9,Version9,/projects/0/einf914/data/20210907_1507_Plate23,790,23,Carrot,A5,001P100N,Y,-,...,None,20210907_1108_Plate23,20210917_1325_Plate23,96,Analysis_1636107306199118080_0_100_Version9/ex...,Analysis_1636107306199118080_0_100_Version9/gl...,Analysis_1636107306199118080_0_100_Version9/ti...,Analysis_1636107306199118080_0_100_Version9/ti...,Analysis_1636107306199118080_0_100_Version9/gl...,"01.01.1970, 01:00:"
2,Analysis_1637745064562052826_0_99_Version9,Version9,/projects/0/einf914/data/20210918_1729_Plate21,792,21,Carrot,A5,001P100N,Y,-,...,None,20210915_1518_Plate21,20210925_1520_Plate21,99,Analysis_1637745064562052826_0_99_Version9/exp...,Analysis_1637745064562052826_0_99_Version9/glo...,Analysis_1637745064562052826_0_99_Version9/tim...,Analysis_1637745064562052826_0_99_Version9/tim...,Analysis_1637745064562052826_0_99_Version9/glo...,"01.01.1970, 01:00:"
3,Analysis_1636104897672902830_0_100_Version9,Version9,/projects/0/einf914/data/20211007_1606_Plate31,800,31,Carrot,A5,001P100N,Y,-,...,None,20210929_0942_Plate31,20211011_1408_Plate31,91,Analysis_1636104897672902830_0_100_Version9/ex...,Analysis_1636104897672902830_0_100_Version9/gl...,Analysis_1636104897672902830_0_100_Version9/ti...,Analysis_1636104897672902830_0_100_Version9/ti...,Analysis_1636104897672902830_0_100_Version9/gl...,"01.01.1970, 01:00:"
4,Analysis_1636044035644400860_0_100_Version9,Version9,/projects/0/einf914/data/20210112_1120_Plate29,22,29,Carrot,A5,001P100N,Y,-,...,None,20210107_1119_Plate29,20210112_2320_Plate29,31,Analysis_1636044035644400860_0_100_Version9/ex...,Analysis_1636044035644400860_0_100_Version9/gl...,Analysis_1636044035644400860_0_100_Version9/ti...,Analysis_1636044035644400860_0_100_Version9/ti...,Analysis_1636044035644400860_0_100_Version9/gl...,"01.01.1970, 01:00:"
5,Analysis_1636044918401523795_0_100_Version9,Version9,/projects/0/einf914/data/20201203_0955_Plate36,80,36,Carrot,A5,001P100N,Y,-,...,None,20201128_1802_Plate36,20201206_2113_Plate36,47,Analysis_1636044918401523795_0_100_Version9/ex...,Analysis_1636044918401523795_0_100_Version9/gl...,Analysis_1636044918401523795_0_100_Version9/ti...,Analysis_1636044918401523795_0_100_Version9/ti...,Analysis_1636044918401523795_0_100_Version9/gl...,"01.01.1970, 01:00:"
6,Analysis_1636043977602188694_0_100_Version9,Version9,/projects/0/einf914/data/20210111_1533_Plate36,10,36,Carrot,A5,001P100N,Y,-,...,None,20210107_1140_Plate36,20210112_2333_Plate36,31,Analysis_1636043977602188694_0_100_Version9/ex...,Analysis_1636043977602188694_0_100_Version9/gl...,Analysis_1636043977602188694_0_100_Version9/ti...,Analysis_1636043977602188694_0_100_Version9/ti...,Analysis_1636043977602188694_0_100_Version9/gl...,"01.01.1970, 01:00:"
7,Analysis_1636044998117269036_0_100_Version9,Version9,/projects/0/einf914/data/20210110_1144_Plate38,76,38,Carrot,A5,001P100N,Y,-,...,None,20210108_1143_Plate38,20210112_2337_Plate38,25,Analysis_1636044998117269036_0_100_Version9/ex...,Analysis_1636044998117269036_0_100_Version9/gl...,Analysis_1636044998117269036_0_100_Version9/ti...,Analysis_1636044998117269036_0_100_Version9/ti...,Analysis_1636044998117269036_0_100_Version9/gl...,"01.01.1970, 01:00:"
8,Analysis_1636367533382636815_0_100_Version9,Version9,/projects/0/einf914/data/20201229

In [3]:
select = analysis_info.loc[analysis_info['Plate']!=22]
# select = analysis_info.loc[analysis_info['Plate']==792]

# analysis_info['Plate']=analysis_info['Plate'].fillna(758)

In [7]:
select

,folder_analysis,version,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,...,Temp,date_begin,date_end,number_timepoints,path_exp,path_global_hypha_info,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis
0,Analysis_1636046826216022929_0_100_Version9,Version9,/projects/0/einf914/data/20211005_0148_Plate39,799,39,Carrot,A5,001P100N,Y,-,...,None,20211001_1620_Plate39,20211011_1227_Plate39,95,Analysis_1636046826216022929_0_100_Version9/ex...,Analysis_1636046826216022929_0_100_Version9/gl...,Analysis_1636046826216022929_0_100_Version9/ti...,Analysis_1636046826216022929_0_100_Version9/ti...,Analysis_1636046826216022929_0_100_Version9/gl...,"01.01.1970, 01:00:"
1,Analysis_1636107306199118080_0_100_Version9,Version9,/projects/0/einf914/data/20210907_1507_Plate23,790,23,Carrot,A5,001P100N,Y,-,...,None,20210907_1108_Plate23,20210917_1325_Plate23,96,Analysis_1636107306199118080_0_100_Version9/ex...,Analysis_1636107306199118080_0_100_Version9/gl...,Analysis_1636107306199118080_0_100_Version9/ti...,Analysis_1636107306199118080_0_100_Version9/ti...,Analysis_1636107306199118080_0_100_Version9/gl...,"01.01.1970, 01:00:"
2,Analysis_1637745064562052826_0_99_Version9,Version9,/projects/0/einf914/data/20210918_1729_Plate21,792,21,Carrot,A5,001P100N,Y,-,...,None,20210915_1518_Plate21,20210925_1520_Plate21,99,Analysis_1637745064562052826_0_99_Version9/exp...,Analysis_1637745064562052826_0_99_Version9/glo...,Analysis_1637745064562052826_0_99_Version9/tim...,Analysis_1637745064562052826_0_99_Version9/tim...,Analysis_1637745064562052826_0_99_Version9/glo...,"01.01.1970, 01:00:"
3,Analysis_1636104897672902830_0_100_Version9,Version9,/projects/0/einf914/data/20211007_1606_Plate31,800,31,Carrot,A5,001P100N,Y,-,...,None,20210929_0942_Plate31,20211011_1408_Plate31,91,Analysis_1636104897672902830_0_100_Version9/ex...,Analysis_1636104897672902830_0_100_Version9/gl...,Analysis_1636104897672902830_0_100_Version9/ti...,Analysis_1636104897672902830_0_100_Version9/ti...,Analysis_1636104897672902830_0_100_Version9/gl...,"01.01.1970, 01:00:"
5,Analysis_1636044918401523795_0_100_Version9,Version9,/projects/0/einf914/data/20201203_0955_Plate36,80,36,Carrot,A5,001P100N,Y,-,...,None,20201128_1802_Plate36,20201206_2113_Plate36,47,Analysis_1636044918401523795_0_100_Version9/ex...,Analysis_1636044918401523795_0_100_Version9/gl...,Analysis_1636044918401523795_0_100_Version9/ti...,Analysis_1636044918401523795_0_100_Version9/ti...,Analysis_1636044918401523795_0_100_Version9/gl...,"01.01.1970, 01:00:"
6,Analysis_1636043977602188694_0_100_Version9,Version9,/projects/0/einf914/data/20210111_1533_Plate36,10,36,Carrot,A5,001P100N,Y,-,...,None,20210107_1140_Plate36,20210112_2333_Plate36,31,Analysis_1636043977602188694_0_100_Version9/ex...,Analysis_1636043977602188694_0_100_Version9/gl...,Analysis_1636043977602188694_0_100_Version9/ti...,Analysis_1636043977602188694_0_100_Version9/ti...,Analysis_1636043977602188694_0_100_Version9/gl...,"01.01.1970, 01:00:"
7,Analysis_1636044998117269036_0_100_Version9,Version9,/projects/0/einf914/data/20210110_1144_Plate38,76,38,Carrot,A5,001P100N,Y,-,...,None,20210108_1143_Plate38,20210112_2337_Plate38,25,Analysis_1636044998117269036_0_100_Version9/ex...,Analysis_1636044998117269036_0_100_Version9/gl...,Analysis_1636044998117269036_0_100_Version9/ti...,Analysis_1636044998117269036_0_100_Version9/ti...,Analysis_1636044998117269036_0_100_Version9/gl...,"01.01.1970, 01:00:"
8,Analysis_1636367533382636815_0_100_Version9,Version9,/projects/0/einf914/data/20201229_0349_Plate40,102,40,Carrot,A5,001P100N,Y,None,...,25,20201226_1224_Plate40,20210105_1945_Plate40,60,Analysis_1636367533382636815_0_100_Version9/ex...,Analysis_1636367533382636815_0_100_Version9/gl...,Analysis_1636367533382636815_0_100_Version9/ti...,Analysis_1636367533382636815_0_100_Version9/ti...,Analysis_1636367533382636815_0_100_Version9/gl...,"01.01.1970, 01:00:"
9,Analysis_1636046622780809853_0_100_Version9,Version9,/projects/0/einf914/data/202011

In [4]:
select = analysis_info.loc[analysis_info['folder_analysis']=="Analysis_1641204283861202846_0_100_Version9"]
select

,folder_analysis,version,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,...,Temp,date_begin,date_end,number_timepoints,path_exp,path_global_hypha_info,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis


In [5]:
select = analysis_info.loc[analysis_info['Plate'].isin([94])]

In [17]:
select = analysis_info.loc[analysis_info['Plate']==792]

In [27]:
time = '10:00'
directory = directory
list_f = [num_hypha,prop_lost_tracks_junction,prop_lost_tracks_tips,prop_lost_tracks_junction,prop_lost_tracks_tips,prop_inconsistent_root,number_of_timepoints_withing_boundaries,number_of_timepoints]
list_args= [None,[1],[1],[10],[10]]+[[]]*len(list_f)
overwrite = True
num_parallel = 5
run_parallel_post('global_plate_post_process.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'global_plate_post_process',cpus = 32,name_job = 'glob_plate')

Submitted batch job 195252


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [7]:
time = '40:00'
directory = directory
list_f = [make_movie_RH_BAS]
list_args= [[]]*len(list_f)
overwrite = True
num_parallel = 30
run_parallel_post('exp_plot.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'global_plate_post_process',cpus = 32,name_job = 'glob_plate',node='fat')

Submitted batch job 345995


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [10]:
time = '20:00'
directory = directory
list_f = []
list_args= [[]]*len(list_f)
overwrite = True
num_parallel = 5
run_parallel_post('make_small_exp.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'global_plate_post_process',cpus = 32,name_job = 'glob_plate')

Submitted batch job 269913


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [ ]:
frames = []
for index, row in select.iterrows():
    ser = pd.read_json(f'{directory}{row["path_global_plate_info"]}',convert_dates=True,typ='series')
    frame = ser.to_frame(index).transpose()
    frames.append(frame)
global_plate_infos = pd.concat(frames)
global_plate_infos

In [8]:
%load_ext autoreload
%autoreload 2
time = '3:40:00'
directory = directory
list_f = [get_L_RH,get_L_BAS,get_num_trunks,get_length,get_area, get_area_separate_connected_components,get_num_tips,get_num_nodes,get_area_study_zone,get_num_tips_study_zone,get_num_nodes_study_zone,get_length_study_zone,is_out_study]
max_ind = 20
incr = 100

list_f = []
list_args = []
fs = [get_density_anastomose_in_ring,get_density_branch_rate_in_ring,get_density_stop_rate_in_ring,get_density_active_tips_in_ring]
# fs = [get_density_anastomose_in_ring]

for f in fs:
    list_f += [f]*max_ind

    list_args+= [{"incr":incr,"i":i, "rh_only" : True} for i in range(max_ind)]
# list_f = [get_L_RH,get_L_BAS]
# list_args = [[]]*len(list_f)
overwrite = False
num_parallel = 30
run_parallel_post('time_plate_post_process.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'time_plate_post_process',cpus = 32,name_job = 'time_plate',node='fat')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Submitted batch job 346145


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [ ]:
frames = []
for index, row in select.iterrows():
    if os.path.isfile(f'{directory}{row["path_time_plate_info"]}'):
        frame = pd.read_json(f'{directory}{row["path_time_plate_info"]}',convert_dates=True).transpose()
        frame.index.name = 't'
        frame.reset_index(inplace=True)
        frames.append(frame)
time_plate_infos = pd.concat(frames,ignore_index=True)
time_plate_infos.loc[time_plate_infos['Plate']==91]

In [17]:
time = '40:00'
directory = directory
# list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f]
# list_f = [get_timestep_anastomosis]
list_f = [get_width_f,get_tot_length_C_f,get_tot_growth_C_f,get_tot_length_pp_f,get_tot_growth_pp_f,get_timestep_stop_growth,get_time_stop_growth,get_time_init_growth,get_mean_speed_growth,get_stop_track,get_timestep_anastomosis,get_timestep_biological_stop_growth]
# list_f = [get_num_branch]
list_f = [gets_out_of_ROI]
list_args= [{}]*len(list_f)
overwrite = False
num_parallel = 32
run_parallel_post('global_hypha_post_process.py', list_f,list_args,[directory,overwrite],select, num_parallel, time,'global_hypha_post_process',cpus = 32,name_job = 'glob_hypha',node='fat')

Submitted batch job 344157


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [ ]:
frames = []
for index, row in select.iterrows():
    if os.path.isfile(f'{directory}{row["path_global_hypha_info"]}'):
        frame = pd.read_json(f'{directory}{row["path_global_hypha_info"]}',convert_dates=True).transpose()
        frame.index.name = 'hypha'
        frame.reset_index(inplace=True)
        frames.append(frame)
global_hypha_info = pd.concat(frames,ignore_index=True)
# time_plate_infos.loc[time_plate_infos['Plate']==91]

In [4]:

# list_f = [get_distance_final_pos,get_timedelta,get_time_since_start,get_speed,get_timestep,get_timestep_init,get_time_init,get_degree,get_width_tip_edge,get_width_root_edge,get_width_average,has_reached_final_pos,in_ROI]
# list_f = [local_density,local_density,local_density]
# list_f = [get_time_since_begin_exp]
list_f = [get_absolute_angle_growth]
list_args= [{}]*len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = False
load_graphs = True
num_parallel = 25
time = '45:00'
for index, row in select.iterrows():
    folder = row['folder_analysis']
    path_time_plate_info = row['path_time_plate_info']
    plate = row['Plate']
    num_cpus = 32
    if os.path.isfile(f'{directory}{path_time_plate_info}'):
        whole_plate_info = pd.read_json(f'{directory}{path_time_plate_info}',
       convert_dates=True).transpose()
        whole_plate_info.index.name = 't'
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post('time_hypha_post_process.py', list_f,list_args,[directory,overwrite,load_graphs],whole_plate_info, num_parallel, time,'time_hypha_post_process',cpus = num_cpus,name_job='time_hypha')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 313579
Submitted batch job 313580
Submitted batch job 313581
Submitted batch job 313582


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 313583
Submitted batch job 313584
Submitted batch job 313585
Submitted batch job 313586


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 313622
Submitted batch job 313623
Submitted batch job 313624
Submitted batch job 313625
Submitted batch job 313626
Submitted batch job 313627
Submitted batch job 313628
Submitted batch job 313629
Submitted batch job 313630


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 313631
Submitted batch job 313632


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [ ]:
frames = []
for index, row in select.iterrows():
    folder = row['folder_analysis']
    path_time_plate_info = row['path_time_plate_info']
    plate = row['Plate']
    if os.path.isfile(f'{directory}{path_time_plate_info}'):
        time_plate_info = pd.read_json(f'{directory}{path_time_plate_info}',
       convert_dates=True).transpose()
        time_plate_info.index.name = 't'
        time_plate_info.reset_index(inplace=True)
        for indexo, rowt in time_plate_info.iterrows():
            t = rowt['t']
            path_hyph_info_t = f'{directory}{folder}/time_hypha_info/hyph_info_{t}.json'
            if os.path.isfile(path_hyph_info_t):
                hyph_info_t = pd.read_json(path_hyph_info_t,
       convert_dates=True).transpose()
                hyph_info_t['Plate']=rowt['Plate']
                hyph_info_t['folder']=folder
                hyph_info_t['t']=rowt['t']
                frames.append(hyph_info_t)
result = pd.concat(frames)

In [ ]:
result